In [1]:
import os

os.environ['HF_HOME'] = "/workspace/cache"

In [2]:
from transformers.trainer_utils import get_last_checkpoint

In [5]:
latest = get_last_checkpoint("/workspace/Qwen-Qwen2.5-14B-Instruct-lora-128-embedding-8k-multipack")
latest

'/workspace/Qwen-Qwen2.5-14B-Instruct-lora-128-embedding-8k-multipack/checkpoint-3260'

In [6]:
from transformers import AutoTokenizer, AutoModelForCausalLM, TextStreamer
from peft import LoraConfig, get_peft_model, AutoPeftModelForCausalLM, PeftModel
import torch

In [7]:
model = AutoModelForCausalLM.from_pretrained('Qwen/Qwen2.5-14B-Instruct', device_map = 'auto',
                                             torch_dtype = torch.bfloat16)
model

Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

Qwen2ForCausalLM(
  (model): Qwen2Model(
    (embed_tokens): Embedding(152064, 5120)
    (layers): ModuleList(
      (0-47): 48 x Qwen2DecoderLayer(
        (self_attn): Qwen2SdpaAttention(
          (q_proj): Linear(in_features=5120, out_features=5120, bias=True)
          (k_proj): Linear(in_features=5120, out_features=1024, bias=True)
          (v_proj): Linear(in_features=5120, out_features=1024, bias=True)
          (o_proj): Linear(in_features=5120, out_features=5120, bias=False)
          (rotary_emb): Qwen2RotaryEmbedding()
        )
        (mlp): Qwen2MLP(
          (gate_proj): Linear(in_features=5120, out_features=13824, bias=False)
          (up_proj): Linear(in_features=5120, out_features=13824, bias=False)
          (down_proj): Linear(in_features=13824, out_features=5120, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): Qwen2RMSNorm((5120,), eps=1e-06)
        (post_attention_layernorm): Qwen2RMSNorm((5120,), eps=1e-06)
      )
    )
    (norm

In [9]:
m = PeftModel.from_pretrained(model, latest, device_map = 'auto', torch_dtype = torch.bfloat16)

In [10]:
m.push_to_hub('mesolitica/Qwen-Qwen2.5-14B-Instruct-lora-128-embedding-8k-multipack', private = True)

/usr/local/lib/python3.10/dist-packages/peft/utils/save_and_load.py:230: UserWarning: Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.
  warnings.warn("Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.")


adapter_model.safetensors:   0%|          | 0.00/5.48G [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/mesolitica/Qwen-Qwen2.5-14B-Instruct-lora-128-embedding-8k-multipack/commit/753f5d3ae5db35256f00824956249b3b0657510e', commit_message='Upload model', commit_description='', oid='753f5d3ae5db35256f00824956249b3b0657510e', pr_url=None, repo_url=RepoUrl('https://huggingface.co/mesolitica/Qwen-Qwen2.5-14B-Instruct-lora-128-embedding-8k-multipack', endpoint='https://huggingface.co', repo_type='model', repo_id='mesolitica/Qwen-Qwen2.5-14B-Instruct-lora-128-embedding-8k-multipack'), pr_revision=None, pr_num=None)

In [11]:
m = m.merge_and_unload()
m

Qwen2ForCausalLM(
  (model): Qwen2Model(
    (embed_tokens): Embedding(152064, 5120)
    (layers): ModuleList(
      (0-47): 48 x Qwen2DecoderLayer(
        (self_attn): Qwen2SdpaAttention(
          (q_proj): Linear(in_features=5120, out_features=5120, bias=True)
          (k_proj): Linear(in_features=5120, out_features=1024, bias=True)
          (v_proj): Linear(in_features=5120, out_features=1024, bias=True)
          (o_proj): Linear(in_features=5120, out_features=5120, bias=False)
          (rotary_emb): Qwen2RotaryEmbedding()
        )
        (mlp): Qwen2MLP(
          (gate_proj): Linear(in_features=5120, out_features=13824, bias=False)
          (up_proj): Linear(in_features=5120, out_features=13824, bias=False)
          (down_proj): Linear(in_features=13824, out_features=5120, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): Qwen2RMSNorm((5120,), eps=1e-06)
        (post_attention_layernorm): Qwen2RMSNorm((5120,), eps=1e-06)
      )
    )
    (norm

In [12]:
tokenizer = AutoTokenizer.from_pretrained(latest)
tokenizer.chat_template = tokenizer.chat_template.replace('You are Qwen, created by Alibaba Cloud', 'You are MaLLaM, created by Mesolitica')

In [13]:
streamer = TextStreamer(tokenizer)

In [14]:
d = [
    {'role': 'user', 'content': 'camne nk selesaikan masalah hutang negara'}
]

with torch.no_grad():
    inputs = tokenizer.apply_chat_template(d, return_tensors = 'pt').to('cuda')
    generate_kwargs = dict(
        input_ids=inputs,
        max_new_tokens=128,
        top_p=0.95,
        top_k=50,
        temperature=0.0,
        do_sample=False,
        repetition_penalty=1.05,
        streamer=streamer
    )
    generation_output = m.generate(**generate_kwargs)

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:633: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


<|im_start|>system
You are MaLLaM, created by Mesolitica. You are a helpful assistant.<|im_end|>
<|im_start|>user
camne nk selesaikan masalah hutang negara<|im_end|>
<|im_start|>assistant
Untuk menyelesaikan masalah hutang negara, beberapa langkah yang boleh diambil ialah:

1. Mengurangkan perbelanjaan kerajaan - Kerajaan perlu mengurangkan perbelanjaan yang tidak perlu dan meningkatkan kecekapan dalam penggunaan dana awam.

2. Meningkatkan pendapatan kerajaan - Kerajaan perlu meningkatkan pendapatan melalui pelbagai cara seperti meningkatkan cukai, memperkenalkan cukai baru, atau meningkatkan hasil daripada


In [15]:
d = [
    {'role': 'user', 'content': 'camne nk selesaikan masalah hutang negara, reply in manglish'}
]

with torch.no_grad():
    inputs = tokenizer.apply_chat_template(d, return_tensors = 'pt').to('cuda')
    generate_kwargs = dict(
        input_ids=inputs,
        max_new_tokens=128,
        top_p=0.95,
        top_k=50,
        temperature=0.0,
        do_sample=False,
        repetition_penalty=1.05,
        streamer=streamer
    )
    generation_output = m.generate(**generate_kwargs)

<|im_start|>system
You are MaLLaM, created by Mesolitica. You are a helpful assistant.<|im_end|>
<|im_start|>user
camne nk selesaikan masalah hutang negara, reply in manglish<|im_end|>
<|im_start|>assistant
Wah lau eh, settle Malaysia's gahmen debt not easy lah, need do many things and take long time. Here got some ideas to help:

1. Make economy grow faster: If economy go up up, then gahmen can make more money from taxes and other things. Must do things like make business easier, invest in education and training, and build more stuff.

2. Cut down on spending: Gahmen must be careful with money lah. Cannot anyhow spend. Must see which things really important, then cut the rest. Also must make sure all the money spent properly, no corruption or


In [16]:
d = [
    {'role': 'user', 'content': 'camne nk selesaikan masalah hutang negara, reply in jawi'}
]

with torch.no_grad():
    inputs = tokenizer.apply_chat_template(d, return_tensors = 'pt').to('cuda')
    generate_kwargs = dict(
        input_ids=inputs,
        max_new_tokens=128,
        top_p=0.95,
        top_k=50,
        temperature=0.0,
        do_sample=False,
        repetition_penalty=1.05,
        streamer=streamer
    )
    generation_output = m.generate(**generate_kwargs)

<|im_start|>system
You are MaLLaM, created by Mesolitica. You are a helpful assistant.<|im_end|>
<|im_start|>user
camne nk selesaikan masalah hutang negara, reply in jawi<|im_end|>
<|im_start|>assistant
مڠورڠكن هوتڠ نڬارا مليسيا اداله توڬس يڠ كومڤليك س يڠ ممرلوكن ڤندكتن ڤلباڬاي روڤا. برايكوت اياله ببراڤ لڠكه يڠ بوليه دأمبيل اونتوق مناڠني ايسو ايت:

١. مڠورڠكن ڤربلنجأن كراجأن: كراجأن بوليه مڠورڠكن ڤربلنجأنڽ دڠن م


In [17]:
d = [
    {'role': 'user', 'content': 'camne nk selesaikan masalah hutang negara, reply in kedah'}
]

with torch.no_grad():
    inputs = tokenizer.apply_chat_template(d, return_tensors = 'pt').to('cuda')
    generate_kwargs = dict(
        input_ids=inputs,
        max_new_tokens=128,
        top_p=0.95,
        top_k=50,
        temperature=0.0,
        do_sample=False,
        repetition_penalty=1.05,
        streamer=streamer
    )
    generation_output = m.generate(**generate_kwargs)

<|im_start|>system
You are MaLLaM, created by Mesolitica. You are a helpful assistant.<|im_end|>
<|im_start|>user
camne nk selesaikan masalah hutang negara, reply in kedah<|im_end|>
<|im_start|>assistant
Nak selesai masalah hutang negara ni, kena buat macam-macam la. Ni aku nak bagitau hang apa yang boleh buat:

1. Kena tambah duit masuk: Kerajaan kena cari cara nak bagi lebih banyak duit masuk negara. Boleh buat macam-macam - tarik orang luar datang labur, buat bisnes baru, kurangkan rasuah, pastu buat kerja elok-elok.

2. Kurangkan duit keluar: Kerajaan pun kena j


In [18]:
m.push_to_hub('mesolitica/malaysian-Qwen-Qwen2.5-14B-Instruct', private = True)

[2024-12-17 10:16:40,073] [INFO] [real_accelerator.py:219:get_accelerator] Setting ds_accelerator to cuda (auto detect)


/usr/bin/ld: cannot find -laio: No such file or directory
collect2: error: ld returned 1 exit status
/usr/bin/ld: cannot find -laio: No such file or directory
collect2: error: ld returned 1 exit status


model-00001-of-00006.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

model-00004-of-00006.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00006-of-00006.safetensors:   0%|          | 0.00/4.73G [00:00<?, ?B/s]

model-00003-of-00006.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

Upload 6 LFS files:   0%|          | 0/6 [00:00<?, ?it/s]

model-00002-of-00006.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00005-of-00006.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/mesolitica/malaysian-Qwen-Qwen2.5-14B-Instruct/commit/6e68be32d4bbb7053f8ce31cba9c030b4cb4fb01', commit_message='Upload Qwen2ForCausalLM', commit_description='', oid='6e68be32d4bbb7053f8ce31cba9c030b4cb4fb01', pr_url=None, repo_url=RepoUrl('https://huggingface.co/mesolitica/malaysian-Qwen-Qwen2.5-14B-Instruct', endpoint='https://huggingface.co', repo_type='model', repo_id='mesolitica/malaysian-Qwen-Qwen2.5-14B-Instruct'), pr_revision=None, pr_num=None)